# Predictions by Llama2 for IBL

<a href="https://colab.research.google.com/github/fuyu-quant/ibl-qlora/blob/main/examples/ibl_llama2_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 以下のリンクからモデルを選んでください(モデルとデータセットはついになっています)
*  https://huggingface.co/fuyu-quant

In [1]:
%%capture
!pip install bitsandbytes==0.40.0 transformers==4.31.0 peft==0.4.0 accelerate==0.21.0 sentencepiece==0.1.99 einops==0.6.1 datasets

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import torch
from peft import PeftModel
from peft import AutoPeftModelForCausalLM

from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('huggingface'))

dataset_name = 'ibl-regression-ver2-branch_easy'
model_name = f'fuyu-quant/{dataset_name}'

In [5]:
model_name = 'fuyu-quant/ibl-regression-ver2-branch_easy'
dataset_name = 'ibl-regression-ver2-branch_easy'

In [12]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-13b-hf')
model = AutoPeftModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

In [16]:
def ibl_output(model, input):
    prompt = f"### Instruction: {input}\n\n### Response: "

    # 推論の実行
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100
            )
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = output.split("### Response: ")[-1]
    return output

## テスト用データのダウンロード

In [15]:
test_dataset = load_dataset(f"fuyu-quant/{dataset_name}", split='test')

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## テスト実行

In [38]:
i = 511

input_data = test_dataset[i]['instruction']
output_data = ibl_output(model, input_data)
print('出力データ:\n', output_data)

print('------')
true_data = test_dataset[i]['output']
print('正解データ:\n', true_data)

出力データ:
  if row["B"] < 4:
    y = 15
else:
    y = 21
------
正解データ:
 if row["A"] < 3:
    y = 15
elif row["C"] < 15:
    y = 21
elif row["B"] < 16:
    y = 27
else:
    y = 9

